# Project: Protein stability prediction

In the project you will try to predict protein stability changes upon point mutations. 
We will use acuumulated data from experimental databases, i.e. the Megascale dataset. A current [pre-print paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10402116/) has already preprocessed the dataset and created homology reduced data splits. We will reuse these. To do so, download the data folder from [here](https://polybox.ethz.ch/index.php/s/txvcb5jKy1A0TbY) and unzip it.  

The data includes measurements of changes in the Gibbs free enrgy ($\Delta \Delta G $). 
This will be the value that you will have to predict for a given protein with a point mutation. 
As input data you can use the protein sequence or a protein embedding retreived from ESM, a state of the art protein model.  

Here we will use protein embeddings computed by ESM as input. 
We provide precomputed embeddings from the last layer of the smallest ESM model. You can adjust the Dataloader's code to load the embedding of the wild type or of the mutaed sequence or both. You can use it however you like. This is just to provide you easy access to embeddings. If you want to compute your own embeddings from other layers or models you can do that, too. 

Below we provide you with a strcuture for the project that you can start with.  
Edit the cells to your liking and add more code to create your final model.

## Imports

In [1]:
import os 
import numpy as np
import pandas as pd
import scipy
import sklearn.metrics as skmetrics

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import lightning as L

import torchmetrics
from torchmetrics.regression import PearsonCorrCoef

## Dataloading

We are using the Megascale dataset. The train, validation and test sets are already predefined.  
As mentioned, we provide embeddings from the last layer of ESM as input. You can access either the wild type or the mutated sequence and you could also further adjsut the embeddings. 
Here we have an embedding representing the complete sequence. It was computed by averaging over the embeddings per residue in the sequence. 

The ``Dataset`` classes return tuples of ``(embedding, ddg_value)``.

In [2]:
# the dataloaders load the tensors from memory one by one, could potentially become a bottleneck

class ProtEmbeddingDataset(Dataset):
    """
    Dataset for the embeddings of the mutated sequences
    You can the get_item() method to return the data in the format you want
    """
    def __init__(self, tensor_folder, csv_file, id_col="name", label_col="ddG_ML"):
        """
        Initialize the dataset
        input at init: 
            tensor_folder: path to the directory with the embeddings we want to use, eg. "/home/data/mega_train_embeddings"
            cvs_file: path to the csv file corresponding to the data, eg. "home/data/mega_train.csv"
        """
        self.tensor_folder = tensor_folder
        self.df = pd.read_csv(csv_file, sep=",")
        # only use the mutation rows
        self.df = self.df[self.df.mut_type!="wt"]
        # get the labels and ids
        self.labels = torch.tensor(self.df[label_col].values)
        self.ids = self.df[id_col].values
        self.wt_names = self.df["WT_name"].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # load embeddings
        # mutation embedding
        tensor_path = os.path.join(self.tensor_folder, self.ids[idx] + ".pt")
        tensor = torch.load(tensor_path)['mean_representations'][6]

        # wildtype embedding, uncomment if you want to use this, too
        #tensor_path_wt = os.path.join(self.tensor_folder, self.wt_names[idx] + ".pt")
        #tensor_wt = torch.load(tensor_path_wt)['mean_representations'][6]

        label = self.labels[idx] # ddG value
        # returns a tuple of the input embedding and the target ddG values
        return tensor, label.float()
    


In [6]:
goOneUp()
os.getcwd()

'/home/course/ProteinStabilityNN'

In [7]:
#go one directory up
def goOneUp():
    path_parent = os.path.dirname(os.getcwd())

    os.chdir(path_parent)
    os.getcwd()

In [8]:
# usage 
# make sure to adjust the paths to where your files are located
dataset_train = ProtEmbeddingDataset('project_data/project_data/mega_train_embeddings', 'project_data/project_data/mega_train.csv')
dataset_val = ProtEmbeddingDataset('project_data/project_data/mega_val_embeddings', 'project_data/project_data/mega_val.csv')
dataset_test = ProtEmbeddingDataset('project_data/project_data/mega_test_embeddings', 'project_data/project_data/mega_test.csv')

dataloader_train = DataLoader(dataset_train, batch_size=1024, shuffle=True, num_workers=16)
dataloader_val = DataLoader(dataset_val, batch_size=512, shuffle=False, num_workers=16)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

## Model architecture and training

Now it's your turn. Create a model trained on the embeddings and the corresponding ddG values.  
Be aware that this is not a classification task, but a regression task. You want to predict a continuous number that is as close to the measured $\Delta \Delta G $ value as possible.
You will need to adjust your architecture and loss accordingly.

Train the model with the predefined dataloaders. And try to improve the model. 
Only test on the test set at the very end, when you have finished fine-tuning you model. 

In [9]:
dataset_test.shape

AttributeError: 'ProtEmbeddingDataset' object has no attribute 'shape'

In [ ]:
dataset_test.__getitem__(8)

In [11]:
# get the output shape of our data after a convolution and pooling of a certain size

def get_conv2d_out_shape(tensor_shape, conv, pool=2):
    # return the new shape of the tensor after a convolution and pooling
    # tensor_shape: (channels, height, width)
    # convolution arguments
    kernel_size = conv.kernel_size
    stride=conv.stride # 2D array
    padding=conv.padding # 2D array
    dilation=conv.dilation # 2D array
    out_channels = conv.out_channels

    height_out = np.floor((tensor_shape[1]+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    #width_out = np.floor((tensor_shape[2]+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)
    
    if pool:
        # adjust dimensions to pooling
        height_out/=pool
        #width_out/=pool
        
    return int(out_channels),int(height_out)#,int(width_out)

In [67]:
# your code
class linModel(nn.Module):
    
    # Network Initialisation
    def __init__(self, params):
        
        super(embedConvModel, self).__init__() #initialize parent pytorch module

        # read parameters
        inputShape = params["inputShape"]
        kernelSize = params["kernelSize"]
        
        #channels_out = params["initial_depth"] 
        #fc1_size = params["fc1_size"]
        
        #### Convolution Layers

        # Max pooling layer
        #self.pool = nn.MaxPool1d(2)
        ##conv layer 1
        # convolution with kernel size 8, goes from three channels to 
        # number defined by initial_depth in params
        #self.conv1 = nn.Conv1d(inputShape[0], inputShape[0], kernel_size=kernelSize[0])#, padding=int((kernelSize[0]-1)/2), padding_mode='zeros')
        #shape in = (3, 256, 256)
        #print(inputShape)
        #outShape = get_conv2d_out_shape(inputShape, self.conv1)#, pool=2)
        #print(outShape)
        #flatSize = outShape[1]# * outShape[2] #768 * 1024
        #print(flatSize)
        #self.fc = nn.Linear(flatSize, 1024)#outShape[1])

    def forward(self,X):
        # our network's forward pass
        
        # Convolution & Pool Layers
        ############# TODO ###############
        # convolution (conv1), then relu, then max pool 
        #print(X.shape)
        X = F.tanh(self.conv1(X))
        #print(X.shape)
        #X = self.pool(X)
        X = torch.flatten(X, 1)
       # print(X.shape)
        
        X = self.fc(X)
        #print(X.shape)
        #####################################
        # return log softmax to fit classification problem, no relu needed
        return X

In [78]:
# your code
class embedConvModel(nn.Module):
    
    # Network Initialisation
    def __init__(self, params):
        
        super(embedConvModel, self).__init__() #initialize parent pytorch module

        # read parameters
        inputShape = params["inputShape"]
        kernelSize = params["kernelSize"]
        
        #channels_out = params["initial_depth"] 
        #fc1_size = params["fc1_size"]
        
        #### Convolution Layers

        # Max pooling layer
        self.pool = nn.MaxPool1d(5)
        ##conv layer 1
        # convolution with kernel size 8, goes from three channels to 
        # number defined by initial_depth in params
        
        
        
        outShape = (1, 1, 1)
        
        self.conv1 = nn.Conv1d(inputShape[0], outShape[0], kernel_size=kernelSize[0])#, padding=int((kernelSize[0]-1)/2), padding_mode='zeros')

        
        outShape = get_conv2d_out_shape(inputShape, self.conv1, pool=5)
        
        
        self.conv2 = nn.Conv1d(outShape[0], outShape[0], kernel_size=kernelSize[1])#, padding=int((kernelSize[0]-1)/2), padding_mode='zeros')

        outShape = get_conv2d_out_shape(outShape, self.conv2, pool=5)
        #print("outShape", outShape)
        self.fc = nn.Linear(outShape[1], 1)#outShape[1])
        
    def forward(self,X):
        # our network's forward pass
        #print("before tanh")
        # Convolution & Pool Layers
        ############# TODO ###############
        # convolution (conv1), then relu, then max pool 
        X = F.leaky_relu(self.conv1(X))
        X = self.pool(X)
        X = F.sigmoid(self.conv2(X)) #all inputs converge to the same values because of sigmoid then pooling 
        X = self.pool(X)
        #print("after tanh")

        #####################################
        # return log softmax to fit classification problem, no relu needed
        return self.fc(X)

In [75]:
class LitMRIModel(L.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        ######## TODO ##########
        # pass our model 
        self.model = model
        #pass the learning rate
        self.lr = learning_rate
        # define loss function
        self.loss_function = nn.MSELoss() #TODO
        # define accuracy metric (torchmetrics)
        #self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        ########################

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        ######### TODO #############
        
        # read from batch
        # TODO
        x,y = batch
        x = torch.unsqueeze(x,1)
        #print(x.shape)
        #print(y)
        # run data through model
        predictions = self.model(x)#.squeeze(1)
        #print(predictions.shape)
       # print(y.shape)
        #print(predictions.shape)
        # compute loss
        # 1 prediction
        loss = self.loss_function(predictions, y)
        #loss = self.loss_function(predictions, y)
        # compute accuracy
        #acc = self.accuracy(predictions, y)
        ##############################

        # logging the values (will appear in progress bar and on dashboard)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        #self.log("train_acc", acc, on_epoch=True, prog_bar=True)

        return loss

    def configure_optimizers(self):
        ############## TODO ################
        # define the optimizer, let's use Adam
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        ####################################
        return optimizer

    def test_step(self, batch, batch_idx):
        # this is the test loop

        ############### TODO #############
        # read from batch
        x,y = batch

        x = torch.unsqueeze(x,1)
        # run data through model
        predictions = self.model(x).squeeze(1)
        
        # compute loss
        #loss = loss(prediction,labels[mask==1])
        loss = self.loss_function(predictions, y)
        # compute accuracy
        #acc = self.accuracy(predictions, y)
        ##############################

        # logging
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss#, acc


    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        ############### TODO #############
        # read from batch
        x,y = batch

        x = torch.unsqueeze(x,1)
        #print(x.shape)
        #print(x.shape)
        #print(y.shape)
        # run data through model
        predictions = self.model(x).squeeze(1).squeeze(1)
        #print(predictions.shape)
        # compute loss
        loss = self.loss_function(predictions, y)
        #loss = self.loss_function(predictions, y)
        # compute accuracy
        #acc = self.accuracy(predictions, y)#torchmetrics.classification.Accuracy(task="MULTICLASS", num_classes=2)
        ##############################

        # logging
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        #self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        return loss 

In [55]:
batch = next(iter(dataloader_train))

In [79]:
# define parameters
params_model={
    "inputShape": (1, batch[0].shape[1]),
    "kernelSize": (7,11)
    }

# define computation hardware approach (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Instantiate the model
cnn_model = embedConvModel(params_model)
# moves the model to GPU if available
cnn_model = cnn_model.to(device)

#channel, batch, weight
#inputShape (1, 1024, 768)

In [45]:
batch[1]

tensor([-6.3381e-02, -1.3375e-04,  5.4088e-01,  ..., -7.0208e-01,
        -6.7147e-01, -7.5817e-01])

In [46]:
torch.unsqueeze(batch[0],0).shape

torch.Size([1, 1024, 768])

In [77]:
# train model
########## TODO #############
# instantiate lightning model with the cnn_model and learning_rate=1e-3
model = LitMRIModel(cnn_model, learning_rate=1e-3)
############################

# instantiate the lightning trainer 
trainer = L.Trainer(max_epochs=20, log_every_n_steps=1)#, callbacks=[FineTuneLearningRateFinder(milestones=(5, 10))])
# train
trainer.fit(model, dataloader_train, dataloader_val)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params
-------------------------------------------------
0 | model         | embedConvModel | 49    
1 | loss_function | MSELoss        | 0     
-------------------------------------------------
49        Trainable params
0         Non-trainable params
49        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: |                                          | 0/? [00:00<?, ?it/s]

Training: |                                                 | 0/? [00:00<?, ?it/s]

Validation: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                               | 0/? [00:00<?, ?it/s]

...

Validation: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                               | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


## Validation and visualization

To get a good feeling of how the model is performing and to compare with literature, compute the Pearson and Spearman correlations.
You can also plot the predictions in a scatterplot. We have added some code for that. 

In [81]:
preds =[]
all_y = []
# save all predictions
for batch in dataloader_val:
    # adjust this to work with your model
    x,y = batch
    x = torch.unsqueeze(x,1)
    y_hat = cnn_model(x.cuda())
    preds.append(y_hat.squeeze().detach().numpy())
    all_y.append(y.detach().numpy())

# concatenate and plot
preds= np.concatenate(preds)
all_y = np.concatenate(all_y)

sns.regplot(x=preds,y=all_y)
plt.xlabel("Predicted ddG")
plt.ylabel("Measured ddG")

# get RMSE, Pearson and Spearman correlation 
print("RMSE:", skmetrics.mean_squared_error(all_y, preds, squared=False))
print("Pearson r:", scipy.stats.pearsonr(preds, all_y))
print("Spearman r:", scipy.stats.spearmanr(preds, all_y))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x142 and 1x1)

In [40]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 1467475), started 1:39:32 ago. (Use '!kill 1467475' to kill it.)